In [1]:
# !pip install transformers deepspeed-mii --upgrade

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
import mii
mii_configs = {"tensor_parallel": 1, "dtype": "fp16"}
mii.deploy(task="text-generation",
           model="bigscience/bloom-560m",
           deployment_name="bloom560m_deployment",
           mii_config=mii_configs)

[2022-11-16 19:23:28,384] [INFO] [deployment.py:87:deploy] ************* MII is using DeepSpeed Optimizations to accelerate your model *************
[2022-11-16 19:23:28,387] [INFO] [server_client.py:219:_initialize_service] MII using multi-gpu deepspeed launcher:
 ------------------------------------------------------------
 task-name .................... text-generation 
 model ........................ bigscience/bloom-560m 
 model-path ................... /tmp/mii_models 
 port ......................... 50050 
 provider ..................... hugging-face 
 ------------------------------------------------------------
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2022-11-16 19:23:29,610] [WARNING] [runner.py:17

In [16]:
generator = mii.mii_query_handle("bloom-560m-mii")

In [17]:
# generator = mii.mii_query_handle("bloom560m_deployment")
result = generator.query({"query": "DeepSpeed is"}, do_sample=False, max_new_tokens=30)
print(result)

------------------------------------------------------
Free memory : 10.848145 (GigaBytes)  
Total memory: 14.560913 (GigaBytes)  
Requested memory: 0.093750 (GigaBytes) 
Setting maximum total tokens (input + output) to 1024 
------------------------------------------------------
response: "DeepSpeed is a new feature in the game. The game is now able to use the new speed to increase the distance between the player and the target. The game"
time_taken: 0.965925932
model_time_taken: -1



In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

In [19]:
start_text = "Testing BLOOM-3B with DeepSpeed MII"
tokens_start_text = len(tokenizer(start_text, return_tensors="pt").input_ids[0])
tokens_start_text

11

In [20]:
import time

new_tokens = 500
gen_length = new_tokens + tokens_start_text
t0 = time.time()
result = generator.query({"query": start_text}, min_length=gen_length, max_length=gen_length)
t1 = time.time()

In [21]:
result

response: "Testing BLOOM-3B with DeepSpeed MII and the results are shown in Fig. 3. The results show that the BLOOM-3B with DeepSpeed MII can achieve a higher accuracy than the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpee

In [22]:
import re

text=str(result)
pattern='(")(.*)(")'

g=re.search(pattern,text)
gen_text = g.group(2)
tokens_gen_text = len(tokenizer(gen_text, return_tensors="pt").input_ids[0])

In [23]:
gen_text

'Testing BLOOM-3B with DeepSpeed MII and the results are shown in Fig. 3. The results show that the BLOOM-3B with DeepSpeed MII can achieve a higher accuracy than the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is higher than that of the BLOOM-3B without DeepSpeed MII. The accuracy of the BLOOM-3B with DeepSpeed MII is h

In [24]:
total_new_tokens_generated = tokens_gen_text - tokens_start_text
throughput = (total_new_tokens_generated) / (t1 - t0)
print(f"Tokens generated: {total_new_tokens_generated}; Time: {t1 - t0:.1f} seconds; Tokens per second: {throughput:.1f}; Latency: {1000 / throughput:.0f} ms")

Tokens generated: 500; Time: 6.8 seconds; Tokens per second: 73.2; Latency: 14 ms


In [14]:
mii.terminate("bloom-560m-mii")

[2022-11-16 19:23:07,242] [INFO] [terminate.py:8:terminate] Terminating server for bloom-560m-mii
[2022-11-16 19:23:08,055] [INFO] [launch.py:318:sigkill_handler] Killing subprocess 4589
[2022-11-16 19:23:08,056] [ERROR] [launch.py:324:sigkill_handler] ['/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python', '-m', 'mii.launch.multi_gpu_server', '--task-name', 'text-generation', '--model', 'bigscience/bloom-560m', '--model-path', '/tmp/mii_models', '--port', '50050', '--ds-optimize', '--provider', 'hugging-face', '--config', 'eyJ0ZW5zb3JfcGFyYWxsZWwiOiAxLCAicG9ydF9udW1iZXIiOiA1MDA1MCwgImR0eXBlIjogImZwMzIiLCAiZW5hYmxlX2N1ZGFfZ3JhcGgiOiBmYWxzZSwgImNoZWNrcG9pbnRfZGljdCI6IG51bGwsICJkZXBsb3lfcmFuayI6IFswXSwgInRvcmNoX2Rpc3RfcG9ydCI6IDI5NTAwLCAiaGZfYXV0aF90b2tlbiI6IG51bGwsICJyZXBsYWNlX3dpdGhfa2VybmVsX2luamVjdCI6IHRydWUsICJwcm9maWxlX21vZGVsX3RpbWUiOiBmYWxzZSwgInNraXBfbW9kZWxfY2hlY2siOiBmYWxzZX0='] exits with return code = -15
